In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
!pip install nibabel

    100% |████████████████████████████████| 3.3MB 5.0MB/s eta 0:00:01
You are using pip version 18.1, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install tensorlayer==1.11.0

    100% |████████████████████████████████| 317kB 18.6MB/s ta 0:00:01
    100% |████████████████████████████████| 13.9MB 4.0MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 42.8MB/s ta 0:00:01
    100% |████████████████████████████████| 25.4MB 2.4MB/s eta 0:00:01
    100% |████████████████████████████████| 5.6MB 12.6MB/s ta 0:00:011
    100% |████████████████████████████████| 5.4MB 13.6MB/s ta 0:00:011
    100% |████████████████████████████████| 31.2MB 1.9MB/s eta 0:00:011
    100% |████████████████████████████████| 13.0MB 5.8MB/s eta 0:00:01
    100% |████████████████████████████████| 3.3MB 19.5MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 32.7MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 37.0MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 50.2MB/s ta 0:00:01
    100% |████████████████████████████████| 2.2MB 20.5MB/s ta 0:00:01
    100% |████████████████████████████████| 1.6MB 28.8MB/s ta 0:00:01
    100% |█████

In [14]:
!pip install kaggle

You are using pip version 18.1, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
import tensorlayer as tl
import numpy as np
import os
import csv
import random
import gc
import pickle
import nibabel as nib

In [13]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [ ]:
!kaggle datasets list

In [18]:
!kaggle datasets download -d sanglequang/brats2018/version/1 -p /media/unet-hdd128

100%|█████████████████████████████████████▊| 3.23G/3.24G [02:25<00:00, 22.6MB/s]
100%|██████████████████████████████████████| 3.24G/3.24G [02:25<00:00, 23.9MB/s]


In [20]:
!unzip /media/unet-hdd128/brats2018.zip -d /media/unet-hdd128/

Archive:  /media/unet-hdd128/brats2018.zip
caution: filename not matched:  /media/unet-hdd128/


In [10]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            15G        299M         15G        704K        114M         15G
Swap:            0B          0B          0B


In [47]:

###============================= SETTINGS ===================================###
DATA_SIZE = 'half' # (small, half or all)

save_dir = "/media/unet-hdd128/data_all/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

HGG_data_path = "/media/unet-hdd128/MICCAI_BraTS_2018_Data_Training/HGG"
LGG_data_path = "/media/unet-hdd128/MICCAI_BraTS_2018_Data_Training/LGG"
survival_csv_path = "/media/unet-hdd128/MICCAI_BraTS_2018_Data_Training/survival_data.csv"
###==========================================================================###

In [15]:
survival_id_list = []
survival_age_list =[]
survival_peroid_list = []

with open(survival_csv_path, 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for idx, content in enumerate(reader):
        survival_id_list.append(content[0])
        survival_age_list.append(float(content[1]))
        survival_peroid_list.append(float(content[2]))

print(len(survival_id_list)) #163

163


In [16]:
if DATA_SIZE == 'all':
    HGG_path_list = tl.files.load_folder_list(path=HGG_data_path)
    LGG_path_list = tl.files.load_folder_list(path=LGG_data_path)
elif DATA_SIZE == 'half':
    HGG_path_list = tl.files.load_folder_list(path=HGG_data_path)[0:100]# DEBUG WITH SMALL DATA
    LGG_path_list = tl.files.load_folder_list(path=LGG_data_path)[0:30] # DEBUG WITH SMALL DATA
elif DATA_SIZE == 'small':
    HGG_path_list = tl.files.load_folder_list(path=HGG_data_path)[0:50] # DEBUG WITH SMALL DATA
    LGG_path_list = tl.files.load_folder_list(path=LGG_data_path)[0:20] # DEBUG WITH SMALL DATA
elif DATA_SIZE == 'smallest':
    HGG_path_list = tl.files.load_folder_list(path=HGG_data_path)[0:20] # DEBUG WITH SMALL DATA
    LGG_path_list = tl.files.load_folder_list(path=LGG_data_path)[0:10] # DEBUG WITH SMALL DATA
    
else:
    exit("Unknow DATA_SIZE")
print(len(HGG_path_list), len(LGG_path_list)) #210 #75

HGG_name_list = [os.path.basename(p) for p in HGG_path_list]
LGG_name_list = [os.path.basename(p) for p in LGG_path_list]

100 30


In [17]:
len(HGG_name_list)

100

In [18]:
survival_id_from_HGG = []
survival_id_from_LGG = []
for i in survival_id_list:
    if i in HGG_name_list:
        survival_id_from_HGG.append(i)
    elif i in LGG_name_list:
        survival_id_from_LGG.append(i)
    else:
        print(i)

print(len(survival_id_from_HGG), len(survival_id_from_LGG)) #163, 0

Brats18_TCIA08_319_1
Brats18_TCIA08_218_1
Brats18_TCIA08_406_1
Brats18_TCIA06_247_1
Brats18_TCIA06_372_1
Brats18_TCIA06_184_1
Brats18_TCIA04_437_1
Brats18_TCIA04_361_1
Brats18_TCIA04_192_1
Brats18_TCIA04_479_1
Brats18_TCIA04_343_1
Brats18_TCIA03_419_1
Brats18_TCIA03_199_1
Brats18_TCIA03_498_1
Brats18_TCIA03_338_1
Brats18_TCIA03_265_1
Brats18_TCIA03_121_1
Brats18_TCIA02_274_1
Brats18_TCIA02_473_1
Brats18_TCIA02_322_1
Brats18_TCIA02_179_1
Brats18_TCIA02_368_1
Brats18_TCIA02_135_1
Brats18_TCIA02_394_1
Brats18_TCIA02_151_1
Brats18_TCIA02_226_1
Brats18_TCIA02_455_1
Brats18_TCIA02_283_1
Brats18_TCIA02_430_1
Brats18_TCIA02_321_1
Brats18_TCIA02_290_1
Brats18_TCIA02_377_1
Brats18_TCIA02_331_1
Brats18_TCIA02_491_1
Brats18_TCIA01_150_1
Brats18_TCIA01_203_1
Brats18_TCIA01_235_1
Brats18_TCIA01_401_1
Brats18_TCIA01_378_1
Brats18_TCIA01_186_1
Brats18_TCIA01_190_1
Brats18_2013_11_1
Brats18_CBICA_AZD_1
Brats18_CBICA_AYA_1
Brats18_CBICA_AXQ_1
Brats18_CBICA_AXL_1
Brats18_CBICA_AWI_1
Brats18_CBICA_AWH_1
B

In [19]:
len(survival_id_list)

163

In [20]:

# use 42 from 210 (in 163 subset) and 15 from 75 as 0.8/0.2 train/dev split

# use 126/42/42 from 210 (in 163 subset) and 45/15/15 from 75 as 0.6/0.2/0.2 train/dev/test split
index_HGG = list(range(0, len(survival_id_from_HGG)))
index_LGG = list(range(0, len(LGG_name_list)))
# random.shuffle(index_HGG)
# random.shuffle(index_HGG)

In [21]:
len(index_HGG)

75

In [22]:
if DATA_SIZE == 'all':
    dev_index_HGG = index_HGG[-84:-42]
    test_index_HGG = index_HGG[-42:]
    tr_index_HGG = index_HGG[:-84]
    dev_index_LGG = index_LGG[-30:-15]
    test_index_LGG = index_LGG[-15:]
    tr_index_LGG = index_LGG[:-30]
elif DATA_SIZE == 'half':
    dev_index_HGG = index_HGG[-30:]  # DEBUG WITH SMALL DATA
    test_index_HGG = index_HGG[-5:]
    tr_index_HGG = index_HGG[:-30]
    dev_index_LGG = index_LGG[-10:]  # DEBUG WITH SMALL DATA
    test_index_LGG = index_LGG[-5:]
    tr_index_LGG = index_LGG[:-10]
elif DATA_SIZE == 'small':
    dev_index_HGG = index_HGG[35:42]   # DEBUG WITH SMALL DATA
    # print(index_HGG, dev_index_HGG)
    # exit()
    test_index_HGG = index_HGG[41:42]
    tr_index_HGG = index_HGG[0:35]
    dev_index_LGG = index_LGG[7:10]    # DEBUG WITH SMALL DATA
    test_index_LGG = index_LGG[9:10]
    tr_index_LGG = index_LGG[0:7]

In [23]:
len(tr_index_HGG)

45

In [24]:
survival_id_dev_HGG = [survival_id_from_HGG[i] for i in dev_index_HGG]
survival_id_test_HGG = [survival_id_from_HGG[i] for i in test_index_HGG]
survival_id_tr_HGG = [survival_id_from_HGG[i] for i in tr_index_HGG]

survival_id_dev_LGG = [LGG_name_list[i] for i in dev_index_LGG]
survival_id_test_LGG = [LGG_name_list[i] for i in test_index_LGG]
survival_id_tr_LGG = [LGG_name_list[i] for i in tr_index_LGG]

survival_age_dev = [survival_age_list[survival_id_list.index(i)] for i in survival_id_dev_HGG]
survival_age_test = [survival_age_list[survival_id_list.index(i)] for i in survival_id_test_HGG]
survival_age_tr = [survival_age_list[survival_id_list.index(i)] for i in survival_id_tr_HGG]

survival_period_dev = [survival_peroid_list[survival_id_list.index(i)] for i in survival_id_dev_HGG]
survival_period_test = [survival_peroid_list[survival_id_list.index(i)] for i in survival_id_test_HGG]
survival_period_tr = [survival_peroid_list[survival_id_list.index(i)] for i in survival_id_tr_HGG]

data_types = ['flair', 't1', 't1ce', 't2']
data_types_mean_std_dict = {i: {'mean': 0.0, 'std': 1.0} for i in data_types}

In [25]:
len(survival_id_dev_HGG)

30

In [ ]:

# preserving_ratio = 0.0
# preserving_ratio = 0.01 # 0.118 removed
# preserving_ratio = 0.05 # 0.213 removed
# preserving_ratio = 0.10 # 0.359 removed

#==================== LOAD ALL IMAGES' PATH AND COMPUTE MEAN/ STD
for i in data_types:
    data_temp_list = []
    for j in HGG_name_list:
        img_path = os.path.join(HGG_data_path, j, j + '_' + i + '.nii.gz')
        img = nib.load(img_path).get_data()
        data_temp_list.append(img)

    for j in LGG_name_list:
        img_path = os.path.join(LGG_data_path, j, j + '_' + i + '.nii.gz')
        img = nib.load(img_path).get_data()
        data_temp_list.append(img)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
    data_temp_list = np.asarray(data_temp_list)
    m = np.mean(data_temp_list)
    s = np.std(data_temp_list)
    data_types_mean_std_dict[i]['mean'] = m
    data_types_mean_std_dict[i]['std'] = s
del data_temp_list
print(data_types_mean_std_dict)

In [26]:
data_types_mean_std_dict = {'flair': {'mean': 55.052796611352356, 'std': 380.9875011565797},
 't1': {'mean': 71.32039103770333, 'std': 366.6561974269796},
 't1ce': {'mean': 80.16341578870278, 'std': 451.33543035257645},
 't2': {'mean': 77.49268785239178, 'std': 443.6208077152802}}

In [27]:
data_types_mean_std_dict

{'flair': {'mean': 55.052796611352356, 'std': 380.9875011565797},
 't1': {'mean': 71.32039103770333, 'std': 366.6561974269796},
 't1ce': {'mean': 80.16341578870278, 'std': 451.33543035257645},
 't2': {'mean': 77.49268785239178, 'std': 443.6208077152802}}

In [28]:
type(data_types_mean_std_dict)

dict

In [29]:
##==================== GET NORMALIZE IMAGES
X_train_input = []
X_train_target = []
# X_train_target_whole = [] # 1 2 4
# X_train_target_core = [] # 1 4
# X_train_target_enhance = [] # 4

X_dev_input = []
X_dev_target = []
# X_dev_target_whole = [] # 1 2 4x
# X_dev_target_core = [] # 1 4
# X_dev_target_enhance = [] # 4

In [30]:
print(" HGG Validation")
for i in survival_id_dev_HGG:
    all_3d_data = []
    for j in data_types:
        img_path = os.path.join(HGG_data_path, i, i + '_' + j + '.nii')
        img = nib.load(img_path).get_data()
        img = (img - data_types_mean_std_dict[j]['mean']) / data_types_mean_std_dict[j]['std']
        img = img.astype(np.float32)
        all_3d_data.append(img)

    seg_path = os.path.join(HGG_data_path, i, i + '_seg.nii')
    seg_img = nib.load(seg_path).get_data()
    seg_img = np.transpose(seg_img, (1, 0, 2))
    for j in range(all_3d_data[0].shape[2]):
        combined_array = np.stack((all_3d_data[0][:, :, j], all_3d_data[1][:, :, j], all_3d_data[2][:, :, j], all_3d_data[3][:, :, j]), axis=2)
        combined_array = np.transpose(combined_array, (1, 0, 2))#.tolist()
        combined_array.astype(np.float32)
        X_dev_input.append(combined_array)

        seg_2d = seg_img[:, :, j]
        # whole = np.zeros_like(seg_2d)
        # core = np.zeros_like(seg_2d)
        # enhance = np.zeros_like(seg_2d)
        # for index, x in np.ndenumerate(seg_2d):
        #     if x == 1:
        #         whole[index] = 1
        #         core[index] = 1
        #     if x == 2:
        #         whole[index] = 1
        #     if x == 4:
        #         whole[index] = 1
        #         core[index] = 1
        #         enhance[index] = 1
        # X_dev_target_whole.append(whole)
        # X_dev_target_core.append(core)
        # X_dev_target_enhance.append(enhance)
        seg_2d.astype(int)
        X_dev_target.append(seg_2d)
    del all_3d_data
    gc.collect()
    print("finished {}".format(i))

 HGG Validation


/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  
/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if sys.path[0] == '':


finished Brats18_CBICA_AXM_1
finished Brats18_CBICA_AXJ_1
finished Brats18_CBICA_AWG_1
finished Brats18_CBICA_ATX_1
finished Brats18_CBICA_ATV_1
finished Brats18_CBICA_ATP_1
finished Brats18_CBICA_ATB_1
finished Brats18_CBICA_ASY_1
finished Brats18_CBICA_ASW_1
finished Brats18_CBICA_ARZ_1
finished Brats18_CBICA_ARW_1
finished Brats18_CBICA_ARF_1
finished Brats18_CBICA_AQV_1
finished Brats18_CBICA_AQU_1
finished Brats18_CBICA_AQR_1
finished Brats18_CBICA_AQQ_1
finished Brats18_CBICA_AQN_1
finished Brats18_CBICA_AQG_1
finished Brats18_CBICA_AQA_1
finished Brats18_CBICA_APY_1
finished Brats18_CBICA_AOZ_1
finished Brats18_CBICA_AOO_1
finished Brats18_CBICA_ANZ_1
finished Brats18_CBICA_ANI_1
finished Brats18_CBICA_ABY_1
finished Brats18_CBICA_ABO_1
finished Brats18_CBICA_ABM_1
finished Brats18_CBICA_ABE_1
finished Brats18_CBICA_ABB_1
finished Brats18_CBICA_AAG_1


In [55]:
print('hell')

hell


In [31]:
print(" LGG Validation")
for i in survival_id_dev_LGG:
    all_3d_data = []
    for j in data_types:
        img_path = os.path.join(LGG_data_path, i, i + '_' + j + '.nii')
        img = nib.load(img_path).get_data()
        img = (img - data_types_mean_std_dict[j]['mean']) / data_types_mean_std_dict[j]['std']
        img = img.astype(np.float32)
        all_3d_data.append(img)

    seg_path = os.path.join(LGG_data_path, i, i + '_seg.nii')
    seg_img = nib.load(seg_path).get_data()
    seg_img = np.transpose(seg_img, (1, 0, 2))
    for j in range(all_3d_data[0].shape[2]):
        combined_array = np.stack((all_3d_data[0][:, :, j], all_3d_data[1][:, :, j], all_3d_data[2][:, :, j], all_3d_data[3][:, :, j]), axis=2)
        combined_array = np.transpose(combined_array, (1, 0, 2))#.tolist()
        combined_array.astype(np.float32)
        X_dev_input.append(combined_array)

        seg_2d = seg_img[:, :, j]
        # whole = np.zeros_like(seg_2d)
        # core = np.zeros_like(seg_2d)
        # enhance = np.zeros_like(seg_2d)
        # for index, x in np.ndenumerate(seg_2d):
        #     if x == 1:
        #         whole[index] = 1
        #         core[index] = 1
        #     if x == 2:
        #         whole[index] = 1
        #     if x == 4:
        #         whole[index] = 1
        #         core[index] = 1
        #         enhance[index] = 1
        # X_dev_target_whole.append(whole)
        # X_dev_target_core.append(core)
        # X_dev_target_enhance.append(enhance)
        seg_2d.astype(int)
        X_dev_target.append(seg_2d)
    del all_3d_data
    gc.collect()
    print("finished {}".format(i))

 LGG Validation


/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  
/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if sys.path[0] == '':


finished Brats18_TCIA13_621_1
finished Brats18_TCIA10_103_1
finished Brats18_TCIA13_615_1
finished Brats18_2013_9_1
finished Brats18_TCIA10_632_1
finished Brats18_TCIA10_442_1
finished Brats18_TCIA13_642_1
finished Brats18_2013_6_1
finished Brats18_2013_29_1
finished Brats18_TCIA09_255_1


In [32]:
X_dev_input = np.asarray(X_dev_input, dtype=np.float32)

In [33]:
X_dev_input.shape[0]

6200

In [34]:
X_dev_input[0].shape

(240, 240, 4)

In [43]:
X_dev_input[20:40,:,:,:].shape

(20, 240, 240, 4)

In [48]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
batch_size = 10
for i in range(0, X_dev_input.shape[0], batch_size):
    path = "X_dev_input_batch_{}.npy".format(i)
    # save to npy file
    save(save_dir  + "X_dev_intput/" path, X_dev_input[i:i+batch_size,:,:,:])

In [50]:
del X_dev_input

In [51]:
X_dev_target = np.asarray(X_dev_target)#, dtype=np.float32)

In [54]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
batch_size = 10
for i in range(0, X_dev_target.shape[0], batch_size):
    path = "X_dev_target_batch_{}.npy".format(i)
    # save to npy file
    save(save_dir + "X_dev_target/" + path, X_dev_target[i:i+batch_size,:,:])

In [55]:
del X_dev_target

In [ ]:
# with open(save_dir + 'train_dev_input_half.pickle', 'wb') as f:
#     pickle.dump(X_dev_input, f, protocol=4)

In [ ]:
# with open(save_dir + 'train_dev_target_half.pickle', 'wb') as f:
#     pickle.dump(X_dev_target, f, protocol=4)

In [57]:
print(" HGG Train")
for i in survival_id_tr_HGG:
    all_3d_data = []
    for j in data_types:
        img_path = os.path.join(HGG_data_path, i, i + '_' + j + '.nii')
        img = nib.load(img_path).get_data()
        img = (img - data_types_mean_std_dict[j]['mean']) / data_types_mean_std_dict[j]['std']
        img = img.astype(np.float32)
        all_3d_data.append(img)

    seg_path = os.path.join(HGG_data_path, i, i + '_seg.nii')
    seg_img = nib.load(seg_path).get_data()
    seg_img = np.transpose(seg_img, (1, 0, 2))
    for j in range(all_3d_data[0].shape[2]):
        combined_array = np.stack((all_3d_data[0][:, :, j], all_3d_data[1][:, :, j], all_3d_data[2][:, :, j], all_3d_data[3][:, :, j]), axis=2)
        combined_array = np.transpose(combined_array, (1, 0, 2))#.tolist()
        combined_array.astype(np.float32)
        X_train_input.append(combined_array)

        seg_2d = seg_img[:, :, j]
        # whole = np.zeros_like(seg_2d)
        # core = np.zeros_like(seg_2d)
        # enhance = np.zeros_like(seg_2d)
        # for index, x in np.ndenumerate(seg_2d):
        #     if x == 1:
        #         whole[index] = 1
        #         core[index] = 1
        #     if x == 2:
        #         whole[index] = 1
        #     if x == 4:
        #         whole[index] = 1
        #         core[index] = 1
        #         enhance[index] = 1
        # X_train_target_whole.append(whole)
        # X_train_target_core.append(core)
        # X_train_target_enhance.append(enhance)
        seg_2d.astype(int)
        X_train_target.append(seg_2d)
    del all_3d_data
    print("finished {}".format(i))
    print(len(X_train_target))

 HGG Train


/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  
/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if sys.path[0] == '':


finished Brats18_TCIA08_167_1
155
finished Brats18_TCIA08_242_1
310
finished Brats18_TCIA08_469_1
465
finished Brats18_TCIA08_280_1
620
finished Brats18_TCIA08_105_1
775
finished Brats18_TCIA08_278_1
930
finished Brats18_TCIA06_165_1
1085
finished Brats18_TCIA06_409_1
1240
finished Brats18_TCIA05_277_1
1395
finished Brats18_TCIA05_478_1
1550
finished Brats18_TCIA04_111_1
1705
finished Brats18_TCIA04_149_1
1860
finished Brats18_TCIA03_474_1
2015
finished Brats18_TCIA03_133_1
2170
finished Brats18_TCIA03_296_1
2325
finished Brats18_TCIA03_257_1
2480
finished Brats18_TCIA03_138_1
2635
finished Brats18_TCIA03_375_1
2790
finished Brats18_TCIA02_471_1
2945
finished Brats18_TCIA02_300_1
3100
finished Brats18_TCIA02_118_1
3255
finished Brats18_TCIA02_314_1
3410
finished Brats18_TCIA02_198_1
3565
finished Brats18_TCIA01_335_1
3720
finished Brats18_TCIA01_411_1
3875
finished Brats18_TCIA01_231_1
4030
finished Brats18_TCIA01_390_1
4185
finished Brats18_TCIA01_499_1
4340
finished Brats18_TCIA01_41

In [58]:
print(" LGG Train")
for i in survival_id_tr_LGG:
    all_3d_data = []
    for j in data_types:
        img_path = os.path.join(LGG_data_path, i, i + '_' + j + '.nii')
        img = nib.load(img_path).get_data()
        img = (img - data_types_mean_std_dict[j]['mean']) / data_types_mean_std_dict[j]['std']
        img = img.astype(np.float32)
        all_3d_data.append(img)

    seg_path = os.path.join(LGG_data_path, i, i + '_seg.nii')
    seg_img = nib.load(seg_path).get_data()
    seg_img = np.transpose(seg_img, (1, 0, 2))
    for j in range(all_3d_data[0].shape[2]):
        combined_array = np.stack((all_3d_data[0][:, :, j], all_3d_data[1][:, :, j], all_3d_data[2][:, :, j], all_3d_data[3][:, :, j]), axis=2)
        combined_array = np.transpose(combined_array, (1, 0, 2))#.tolist()
        combined_array.astype(np.float32)
        X_train_input.append(combined_array)

        seg_2d = seg_img[:, :, j]
        # whole = np.zeros_like(seg_2d)
        # core = np.zeros_like(seg_2d)
        # enhance = np.zeros_like(seg_2d)
        # for index, x in np.ndenumerate(seg_2d):
        #     if x == 1:
        #         whole[index] = 1
        #         core[index] = 1
        #     if x == 2:
        #         whole[index] = 1
        #     if x == 4:
        #         whole[index] = 1
        #         core[index] = 1
        #         enhance[index] = 1
        # X_train_target_whole.append(whole)
        # X_train_target_core.append(core)
        # X_train_target_enhance.append(enhance)
        seg_2d.astype(int)
        X_train_target.append(seg_2d)
    del all_3d_data
    print("finished {}".format(i))

 LGG Train


/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  
/home/azureuser/.pyenv/versions/3.6.9/envs/unet-env-3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if sys.path[0] == '':


finished Brats18_TCIA12_466_1
finished Brats18_TCIA09_462_1
finished Brats18_TCIA10_310_1
finished Brats18_TCIA10_109_1
finished Brats18_TCIA10_299_1
finished Brats18_TCIA10_241_1
finished Brats18_TCIA10_629_1
finished Brats18_TCIA13_653_1
finished Brats18_2013_15_1
finished Brats18_TCIA09_493_1
finished Brats18_TCIA09_620_1
finished Brats18_TCIA10_413_1
finished Brats18_TCIA13_645_1
finished Brats18_2013_0_1
finished Brats18_TCIA10_282_1
finished Brats18_TCIA09_177_1
finished Brats18_2013_24_1
finished Brats18_TCIA10_130_1
finished Brats18_TCIA10_346_1
finished Brats18_TCIA10_639_1


In [59]:
len(X_train_input)

10075

In [63]:
len(X_train_input[5:10])

5

In [64]:
a = np.asarray(X_train_input[5:10], dtype=np.float32)

In [65]:
a.shape

(5, 240, 240, 4)

In [ ]:
X_train_input.shape

In [ ]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
batch_size = 10
for i in range(0, len(X_train_input), batch_size):
    path = "X_train_input_batch_{}.npy".format(i)
    data = np.asarray(X_train_input[i:i+batch_size], dtype=np.float32)
    # save to npy file
    save(save_dir + "X_train_input/" + path, data)

In [ ]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
batch_size = 10
for i in range(0, len(X_train_target), batch_size):
    path = "X_train_target_batch_{}.npy".format(i)
    data = np.asarray(X_train_target[i:i+batch_size], dtype=np.float32)
    # save to npy file
    save(save_dir + "X_train_target/" + path, data)

In [ ]:
# with open(save_dir + 'train_input_half.pickle', 'wb') as f:
#     pickle.dump(X_train_input, f, protocol=4)

In [ ]:
# with open(save_dir + 'train_target.pickle', 'wb') as f:
#     pickle.dump(X_train_target, f, protocol=4)